# sen2like in openEO

### sen2like

The Sen2Like, demonstration processor has been developed by ESA in the framework of the EU Copernicus programme (https://www.copernicus.eu/). The main goal of Sen2Like is to generate Sentinel-2 like harmonised/fused surface reflectances with higher periodicity by integrating additional compatible optical mission sensors.

https://github.com/senbox-org/sen2like

### openEO

openEO provides an option to run the sen2like processor on the requested extent on an openEO backend. The backend takes care of getting the required input data and the computational resources to run sen2like on. openEO users can start sen2like jobs on the backend and download the results as in the example given above.

https://openeo.cloud/

### Results

The outputs of sen2like are stored in Sentinel 2 - like .SAFE folders. In openEO, users can choose between the output formats NetCDF and GeoTiff. For the sen2like processing in openEO, it is now additionally possible to download the original .SAFE folders. Therefore, the export_original_files parameter needs to be set to True. (See below)

In [1]:
import openeo
from openeo.rest.datacube import PGNode, THIS

In [2]:
eodc = "https://openeo-dev.eodc.eu/openeo/1.1.0/"

In [3]:
conn = openeo.connect(eodc).authenticate_oidc()

Authenticated using refresh token.


In [4]:
conn.describe_process("sen2like")

{'id': 'sen2like',
 'summary': 'Run the Sen2like processor ',
 'description': 'The Sen2like processor generates Sentinel-2-like harmonised or fused surface reflectances with higher periodicity by integrating additional compatible optical mission sensors. The source code is available at https://github.com/senbox-org/sen2like.',
 'categories': ['cubes'],
 'parameters': [{'name': 'data',
   'description': 'The datacube for which to generate L2F & L2H level data.',
   'optional': False,
   'deprecated': False,
   'experimental': False,
   'schema': {'type': 'object', 'subtype': 'datacube', 'deprecated': False}},
  {'name': 'target_product',
   'description': "Per default the target_product is set to 'L2F' to generate the fused sen2like product. Set to 'L2H' to generate the harmonised product.",
   'optional': True,
   'deprecated': False,
   'experimental': False,
   'default': 'L2F',
   'schema': {'type': 'string', 'deprecated': False}},
  {'name': 'cloud_cover',
   'description': 'The Cloud coverage parameter (percentage) to filter MGRS image tile contaminates with strong cloud coverage. ',
   'optional': True,
   'deprecated': False,
   'experimental': False,
   'default': 50,
   'schema': {'type': ['number', 'null'], 'deprecated': False}},
  {'name': 'export_original_files',
   'description': 'Sen2like produces outputs following the Sentinel 2 .SAFE convention. To enable the download for these, set this to True. The results will then include .zip files with the .SAFE folders inside. ',
   'optional': True,
   'deprecated': False,
   'experimental': False,
   'default': False,
   'schema': {'type': 'boolean', 'deprecated': False}}],
 'returns': {'description': 'Sen2like enriched data cube for further processing. For further openEO processing on the sen2like datacube, the datacube is loaded in the Equi7 coordinate reference system, with the spatio-temporal extent of the initially requested data. The resolution is based on the highest resolution of the requested bands, where bands with lower resolution are resampled to the higher resolution respectively. ',
  'schema': {'type': 'object', 'subtype': 'datacube', 'deprecated': False}},
 'deprecated': False,
 'experimental': True,
 'links': [{'rel': 'about',
   'href': 'https://openeo.org/documentation/1.0/datacubes.html',
   'title': 'Data Cubes explained in the openEO documentation'},
  {'rel': 'about',
   'href': 'https://github.com/senbox-org/sen2like',
   'title': 'Source code for the sen2like processor'}]}

### Details in openEO's sen2like processing 

To create an openeo process graph, we define the spatial and temporal extent, we want to process. The bands which can be processed by sen2like are:
- 'B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B8A', 'B11','B12'

For sen2like, we additionally set the parameters: 
- target_product, which can be L2F or L2H
- the cloud_cover, which is between 1 and 100. The lower the value, the lower the cloud cover for the files must be - files with higher cloud cover are ignored.
- export_original_files: can be True or False - if True, the .SAFE folders are an output of the openEO process as well.

In [5]:
collection      = 'SENTINEL2_L1C'
spatial_extent  = {"west": 15.6, "east": 15.7, "south": 46.5, "north": 46.6}
temporal_extent = ["2023-07-01", "2023-09-30"]
bands = ["B02", "B03", "B04"]

S2 = conn.load_collection(collection, spatial_extent=spatial_extent, temporal_extent=temporal_extent, bands=bands)
sen2like = S2.process('sen2like', {
    'data': THIS,
    'target_product': 'L2F', 
    'export_original_files': True,
    'cloud_cover': 50}) 

In [6]:
sen2like.flat_graph()

{'loadcollection1': {'process_id': 'load_collection',
  'arguments': {'bands': ['B02', 'B03', 'B04'],
   'id': 'SENTINEL2_L1C',
   'spatial_extent': {'west': 15.6,
    'east': 15.7,
    'south': 46.5,
    'north': 46.6},
   'temporal_extent': ['2023-07-01', '2023-09-30']}},
 'sen2like1': {'process_id': 'sen2like',
  'arguments': {'cloud_cover': 50,
   'data': {'from_node': 'loadcollection1'},
   'export_original_files': True,
   'target_product': 'L2F'},
  'result': True}}

We create and start the openEO batch job:

In [7]:
job = conn.create_job(sen2like.flat_graph()).start_job()

In [8]:
job

<BatchJob job_id='41cee886-f2b5-4f52-b9b5-89317644bda2'>

In [9]:
job.status()

'finished'

When the job is finished, the results can be downloaded. 

In [10]:
results = job.get_results()
metadata = results.get_metadata()

Sen2like produces .SAFE files, which are zipped in order to enable the download. These can be found in the metadata assets: E.g.: 

In [13]:
metadata['assets']['S2A_MSIL2F_20230815T095031_N0404_R079_T33TWM_20240308T095024.SAFE']

{'href': 'https://openeo.eodc.eu/openeo/1.1.0/files/db4c5212-1900-4b44-98eb-beb201dd7efe/OPENEO/results/S2A_MSIL2F_20230815T095031_N0404_R079_T33TWM_20240308T095024.zip?Expires=1710505168&KeyName=SIGN_KEY_1&UserId=2c3c496d-935a-4f39-b246-da2da4e07a6a&Signature=ADrPkUm8B_q_hICGC00Aw2Ec2KI=',
 'title': 'S2A_MSIL2F_20230815T095031_N0404_R079_T33TWM_20240308T095024',
 'proj:epsg': 32633,
 'roles': ['data']}

### Auxiliary data (openEO takes care of the datasets)

Using openEO to process sen2like, the user does not have to take care of getting the input datasets - as they are provided by the backend. 

Not only does the backend provide you with the Sentinel 2 and Landsat 8 & 9 data, that is required, but it also includes all auxiliary data, needed by sen2like, automatically. Such as:

- CAMS data:
    Available data includes information about
    - Total Aerosol Optical Depth at 550nm
    - Total column water vapour
    - Mean sea level pressure
    - GEMS Total column ozone
- Sentinel 2 reference images:
    - Original source: https://s2gri.csgroup.space/
- Digital elevation model: 
    - Original source: https://sentinels.copernicus.eu/web/sentinel/-/copernicus-dem-new-direct-data-download-access
